In [1]:
from io import open
import unicodedata
import string
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

from google.colab import drive
drive.mount('/content/drive')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
cuda


# 1. Возьмите англо-русскую пару фраз
www.manythings.org....org/anki/

In [2]:
!tail drive/MyDrive/eng-rus.txt

We need to uphold laws against discrimination — in hiring, and in housing, and in education, and in the criminal justice system. That is what our Constitution and our highest ideals require.	Нам нужно отстаивать законы против дискриминации при найме на работу, в жилищной сфере, в сфере образования и правоохранительной системе. Этого требуют наша Конституция и высшие идеалы.	CC-BY 2.0 (France) Attribution: tatoeba.org #5762728 (BHO) & #6390439 (odexed)
I've heard that you should never date anyone who is less than half your age plus seven. Tom is now 30 years old and Mary is 17. How many years will Tom need to wait until he can start dating Mary?	Я слышал, что никогда не следует встречаться с кем-то вдвое младше вас плюс семь лет. Тому 30 лет, a Мэри 17. Сколько лет Тому нужно ждать до тех пор, пока он сможет начать встречаться с Мэри?	CC-BY 2.0 (France) Attribution: tatoeba.org #10068197 (CK) & #10644473 (notenoughsun)
I do have one final ask of you as your president, the same thing I a

In [3]:
SOS_token = 0
EOS_token = 1

class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: 'SOS', 1: 'EOS'}
        self.n_words = 2

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [4]:
# Turn a Unicode string to plain ASCII, thanks to
# http://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters
def normalizeString(s, alphabet='latin'):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    if alphabet == 'latin':
        s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    if alphabet == 'cyrillic':
        s = re.sub(r"[^а-яёА-ЯЁ.!?]+", r" ", s)
    return s

In [5]:
def readLangs(lang1, lang2, reverse=False):
    print("Reading lines...")

    # Read the file and split into lines
    lines = open('drive/MyDrive/%s-%s.txt' % (lang1, lang2), encoding='utf-8').read().strip().split('\n')

    # Split every line into pairs and normalize
    pairs = [[normalizeString(l.split('\t')[0]), normalizeString(l.split('\t')[1], alphabet='cyrillic')] for l in lines]

    # Reverse pairs, make Lang instances
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    else:
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)
    return input_lang, output_lang, pairs

In [6]:
MAX_LENGTH = 10

eng_prefixes = (
    "i am ", "i m ",
    "he is", "he s ",
    "she is", "she s",
    "you are", "you re ",
    "we are", "we re ",
    "they are", "they re "
)

def filterPair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and \
        len(p[1].split(' ')) < MAX_LENGTH and \
        p[1].startswith(eng_prefixes)

def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

In [7]:
def prepareData(lang1, lang2, reverse=False):
    input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse)
    print("Read %s sentence pairs" % len(pairs))
    pairs = filterPairs(pairs)
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs

input_lang, output_lang, pairs = prepareData('eng', 'rus', True)
print(random.choice(pairs))

Reading lines...
Read 487600 sentence pairs
Trimmed to 28240 sentence pairs
Counting words...
Counted words:
rus 10116
eng 4289
['рад что ты решил приити .', 'i m glad you decided to come .']


The Encoder
-----------





In [8]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output, hidden = self.gru(embedded, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

The Decoder
-----------




In [9]:
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [10]:
def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]

def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)

def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    output_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, output_tensor)

In [11]:
teacher_forcing_ratio = 0.5

def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_token]], device=device)
    # print(encoder_hidden.shape)
    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        for di in range(target_length):
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]

    else:
        for di in range(target_length):
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
            loss += criterion(decoder_output, target_tensor[di])
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()
            if decoder_input.item() == EOS_token:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

In [12]:
import time
import math

def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [13]:
def trainIters(encoder, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0
    plot_loss_total = 0

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    training_pairs = [tensorsFromPair(random.choice(pairs)) for i in range(n_iters)]
    criterion = nn.NLLLoss()

    for iter in range(1, n_iters + 1):
        training_pair = training_pairs[iter-1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]

        loss = train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters), iter, iter / n_iters * 100, print_loss_avg))

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

In [14]:
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei], encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)
        decoder_hidden = encoder_hidden
        decoded_words = []

        for di in range(MAX_LENGTH):
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_lang.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words

In [15]:
def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words = evaluate(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

# 2. Обучите на них seq2seq по аналогии с занятием. Оцените полученное качество

In [ ]:
hidden_size=256
encoder1 = EncoderRNN(input_lang.n_words, hidden_size).to(device)
decoder1 = DecoderRNN(hidden_size, output_lang.n_words).to(device)

trainIters(encoder1, decoder1, 75000, print_every=5000)

1m 9s (- 16m 14s) (5000 6%) 3.0977
2m 12s (- 14m 18s) (10000 13%) 2.6155
3m 15s (- 13m 1s) (15000 20%) 2.3653
4m 18s (- 11m 49s) (20000 26%) 2.1429
5m 20s (- 10m 41s) (25000 33%) 2.0106
6m 24s (- 9m 36s) (30000 40%) 1.8699
7m 27s (- 8m 31s) (35000 46%) 1.7446
8m 31s (- 7m 27s) (40000 53%) 1.6634
9m 34s (- 6m 22s) (45000 60%) 1.5596
10m 38s (- 5m 19s) (50000 66%) 1.4880
11m 41s (- 4m 15s) (55000 73%) 1.4330
12m 44s (- 3m 11s) (60000 80%) 1.3706
13m 48s (- 2m 7s) (65000 86%) 1.3088
14m 53s (- 1m 3s) (70000 93%) 1.2457
15m 57s (- 0m 0s) (75000 100%) 1.1997


In [ ]:
evaluateRandomly(encoder1, decoder1)

> я не буду тебя слушать .
= i m not going to listen to you .
< i m not going to stop you . <EOS>

> прошу прощения за беспокоиство .
= i m sorry for the trouble .
< i m sorry to be . <EOS>

> я выживу .
= i m going to survive .
< i m a . . <EOS>

> ты одна меня понимаешь .
= you re the only one who understands me .
< you re the only one who understands me . <EOS>

> я не предлагаю чтобы ты это делал .
= i m not suggesting you do that .
< i m not suggesting you do that . <EOS>

> она медленно скрылась в туманном лесу .
= she slowly disappeared into the foggy forest .
< she is unable to the her . . <EOS>

> я собираюсь задать тебе вопрос .
= i m going to ask you a question .
< i m going to take a a . <EOS>

> я уже не так богат как раньше .
= i m not as rich as i was .
< i m not as rich as i i . .

> вы очень хорошии .
= you re very good .
< you re very good . <EOS>

> я рад что мы делаем это вместе .
= i m glad we re doing this together .
< i m glad we did it . <EOS>



# 3. Попробуйте добавить +1 рекуррентный слой в encoder и decoder

In [16]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, num_layers=2)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output, hidden = self.gru(embedded, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(2, 1, self.hidden_size, device=device)


class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, num_layers=2)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        return torch.zeros(2, 1, self.hidden_size, device=device)

In [19]:
hidden_size=256
encoder2 = EncoderRNN(input_lang.n_words, hidden_size).to(device)
decoder2 = DecoderRNN(hidden_size, output_lang.n_words).to(device)

trainIters(encoder2, decoder2, 75000, print_every=5000)

1m 14s (- 17m 26s) (5000 6%) 3.1224
2m 25s (- 15m 43s) (10000 13%) 2.6900
3m 36s (- 14m 26s) (15000 20%) 2.4474
4m 48s (- 13m 12s) (20000 26%) 2.2334
5m 59s (- 11m 59s) (25000 33%) 2.0723
7m 11s (- 10m 47s) (30000 40%) 1.9376
8m 22s (- 9m 34s) (35000 46%) 1.7784
9m 32s (- 8m 20s) (40000 53%) 1.6951
10m 43s (- 7m 8s) (45000 60%) 1.6107
11m 52s (- 5m 56s) (50000 66%) 1.5227
13m 2s (- 4m 44s) (55000 73%) 1.4358
14m 13s (- 3m 33s) (60000 80%) 1.3755
15m 23s (- 2m 22s) (65000 86%) 1.2853
16m 33s (- 1m 10s) (70000 93%) 1.2513
17m 43s (- 0m 0s) (75000 100%) 1.2064


In [20]:
evaluateRandomly(encoder2, decoder2)

> я астроном .
= i m an astronomer .
< i m an . . <EOS>

> у меня зависимость .
= i m addicted .
< i m going . <EOS>

> я рад что том это сделал .
= i m glad tom did that .
< i m glad tom did that . <EOS>

> меня уволят .
= i m going to get fired .
< i m going to get . <EOS>

> я не впечатлен .
= i m not impressed .
< i m not impressed . <EOS>

> мы уезжаем из австралии в следующем месяце .
= we re leaving australia next month .
< we re leaving for australia next month . <EOS>

> я пишу еще один рассказ .
= i m writing another story .
< i m going to love with with . <EOS>

> я здесь в отпуске .
= i m here on vacation .
< i m here on vacation . <EOS>

> я не такои быстрыи как том .
= i m not as fast as tom .
< i m not as as as tom . <EOS>

> я здесь по делам .
= i am here on business .
< i m here on business . <EOS>



# 4. Попробуйте заменить GRU ячейки на lstm-ячейки

In [17]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.lstm = nn.LSTM(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output, hidden = self.lstm(embedded, hidden)
        return output, hidden

    def initHidden(self):
        return (torch.zeros(1, 1, self.hidden_size, device=device), torch.zeros(1, 1, self.hidden_size, device=device))


class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.lstm = nn.LSTM(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        output, hidden = self.lstm(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        return (torch.zeros(1, 1, self.hidden_size, device=device), torch.zeros(1, 1, self.hidden_size, device=device))

In [18]:
hidden_size=256
encoder3 = EncoderRNN(input_lang.n_words, hidden_size).to(device)
decoder3 = DecoderRNN(hidden_size, output_lang.n_words).to(device)

trainIters(encoder3, decoder3, 75000, print_every=5000)

1m 9s (- 16m 17s) (5000 6%) 3.2141
2m 12s (- 14m 23s) (10000 13%) 2.7708
3m 18s (- 13m 15s) (15000 20%) 2.5469
4m 31s (- 12m 27s) (20000 26%) 2.3953
5m 38s (- 11m 16s) (25000 33%) 2.2137
6m 43s (- 10m 5s) (30000 40%) 2.1037
7m 46s (- 8m 53s) (35000 46%) 1.9847
8m 50s (- 7m 43s) (40000 53%) 1.8804
9m 52s (- 6m 35s) (45000 60%) 1.7888
10m 58s (- 5m 29s) (50000 66%) 1.7055
12m 1s (- 4m 22s) (55000 73%) 1.6159
13m 4s (- 3m 16s) (60000 80%) 1.5629
14m 9s (- 2m 10s) (65000 86%) 1.5195
15m 11s (- 1m 5s) (70000 93%) 1.4672
16m 15s (- 0m 0s) (75000 100%) 1.4249


In [19]:
evaluateRandomly(encoder3, decoder3)

> они все голодны .
= they re all hungry .
< they re all hungry . <EOS>

> мы играем в карты .
= we re playing cards .
< we re in the garden . <EOS>

> я с тобои откровенен .
= i m being candid with you .
< i m going with you . <EOS>

> я не иду домои .
= i m not coming home .
< i m not coming home . <EOS>

> я замешана .
= i m involved .
< i m very . <EOS>

> сегодня мне гораздо лучше .
= i m feeling much better today .
< i m feeling much better today . <EOS>

> она хорошо плавает .
= she is a good swimmer .
< she is a at . . <EOS>

> ты странная девушка .
= you re a strange girl .
< you re a strange girl . <EOS>

> ты побеждаешь .
= you re winning .
< you re a . <EOS>

> они ужасно любопытные .
= they re awfully nosy .
< they re trying to . <EOS>



# 5. +1 рекуррентный слой в encoder и decoder на lstm-ячейках

In [20]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.lstm = nn.LSTM(hidden_size, hidden_size, num_layers=2)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output, hidden = self.lstm(embedded, hidden)
        return output, hidden

    def initHidden(self):
        return (torch.zeros(2, 1, self.hidden_size, device=device), torch.zeros(2, 1, self.hidden_size, device=device))


class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.lstm = nn.LSTM(hidden_size, hidden_size, num_layers=2)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        output, hidden = self.lstm(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        return (torch.zeros(2, 1, self.hidden_size, device=device), torch.zeros(2, 1, self.hidden_size, device=device))

In [21]:
hidden_size=256
encoder4 = EncoderRNN(input_lang.n_words, hidden_size).to(device)
decoder4 = DecoderRNN(hidden_size, output_lang.n_words).to(device)

trainIters(encoder4, decoder4, 75000, print_every=5000)

1m 15s (- 17m 43s) (5000 6%) 3.3629
2m 31s (- 16m 23s) (10000 13%) 2.8616
3m 46s (- 15m 4s) (15000 20%) 2.6851
5m 0s (- 13m 47s) (20000 26%) 2.5414
6m 17s (- 12m 34s) (25000 33%) 2.3656
7m 32s (- 11m 18s) (30000 40%) 2.2365
8m 47s (- 10m 2s) (35000 46%) 2.1316
10m 2s (- 8m 47s) (40000 53%) 2.0149
11m 17s (- 7m 31s) (45000 60%) 1.9028
12m 31s (- 6m 15s) (50000 66%) 1.8380
13m 47s (- 5m 0s) (55000 73%) 1.7776
15m 2s (- 3m 45s) (60000 80%) 1.6992
16m 16s (- 2m 30s) (65000 86%) 1.6000
17m 31s (- 1m 15s) (70000 93%) 1.5568
18m 46s (- 0m 0s) (75000 100%) 1.4723


In [22]:
evaluateRandomly(encoder4, decoder4)

> я эффективен .
= i m efficient .
< i m a . <EOS>

> я увижу ее завтра .
= i m going to see her tomorrow .
< i m going to him him . <EOS>

> бог благословил меня хорошим здоровьем .
= i m blessed with good health .
< i m worried of your father . <EOS>

> я стараюсь не думать об этом .
= i m trying not to think about that .
< i m trying to to about about that . <EOS>

> они азиаты .
= they re asian .
< they re going . <EOS>

> вы высокомерны .
= you re arrogant .
< you re hopeless . <EOS>

> я так от всего этого устала .
= i m so sick of all this .
< i m so tired of that . <EOS>

> я самыи счастливыи человек на земле .
= i am the happiest man on earth .
< i m the the man person . <EOS>

> я уверена что у тебя все получится .
= i m sure that you ll succeed .
< i m sure that you will succeed . <EOS>

> мы никогда не оставляем надежду .
= we re never giving up hope .
< we re never going to . . <EOS>

